<a href="https://colab.research.google.com/github/DulanDias/DNA-Sequence-Alignment/blob/master/Resolving_the_Semantic_Heterogeneity_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Resolving the Semantic Heterogeneity Problem**

In [0]:
import pandas as pd
import numpy as np

import array
import dateutil.parser

#### **1. Load and Understand the Data from Multiple Data Sources**

In [13]:
df_1 = pd.read_csv("https://raw.githubusercontent.com/DulanDias/semantic-heterogeneity-problem/master/data_1.csv")
df_1.head(10)

,Date,24 hr New Snow,Season Snowfall Total,Base Depth
0,2-Jan-17,8 cm,8 cm,74 cm
1,4-Jan-17,8 cm,16 cm,71 cm
2,5-Jan-17,8 cm,24 cm,76 cm
3,8-Jan-17,10 cm,34 cm,81 cm
4,10-Jan-17,20 cm,54 cm,79 cm
5,11-Jan-17,48 cm,102 cm,99 cm
6,12-Jan-17,10 cm,112 cm,109 cm
7,20-Jan-17,10 cm,122 cm,94 cm
8,21-Jan-17,3 cm,125 cm,97 cm
9,22-Jan-17,15 cm,140 cm,94 cm


In [14]:
df_1.describe()

,Date,24 hr New Snow,Season Snowfall Total,Base Depth
count,131,131,131,131
unique,131,18,129,35
top,11-Dec-16,5 cm,24 cm,56 cm
freq,1,19,2,17


In [15]:
df_2 = pd.read_csv("https://raw.githubusercontent.com/DulanDias/semantic-heterogeneity-problem/master/data_2.csv")
df_2.head(10)

,Date,24 hr New Snow,Season Snowfall Total,Base Depth
0,2-Jan-17,0.13 m,0.13 m,1.52 m
1,3-Jan-17,0. 46 m,0.59 m,1.88 m
2,4-Jan-17,0.05 m,0.64 m,1.88 m
3,5-Jan-17,0.28 m,0.92 m,1.85 m
4,8-Jan-17,0.08 m,1 m,1.85 m
5,9-Jan-17,0.13 m,1.13 m,1.85 m
6,10-Jan-17,0.28 m,1.41 m,2.08 m
7,11-Jan-17,0.18 m,1.59 m,2.08 m
8,12-Jan-17,0.15 m,1.74 m,2.29 m
9,13-Jan-17,0.10 m,1.84 m,2.34 m


In [16]:
df_2.describe()

,Date,24 hr New Snow,Season Snowfall Total,Base Depth
count,118,118,118,118
unique,118,22,112,68
top,17-Jan-16,0.03 m,1.74 m,1.85 m
freq,1,19,2,5


In [17]:
df_3 = pd.read_csv("https://raw.githubusercontent.com/DulanDias/semantic-heterogeneity-problem/master/data_3.csv")
df_3.head(10)

,Date,24 hr New Snow,Season Snowfall Total,Base Depth
0,1/1/2017,3 cm,3 cm,91 cm
1,1/2/2017,8 cm,11 cm,94 cm
2,1/3/2017,3 cm,14 cm,91 cm
3,1/4/2017,3 cm,17 cm,94 cm
4,1/5/2017,10 cm,27 cm,99 cm
5,1/6/2017,13 cm,40 cm,109 cm
6,1/8/2017,3 cm,43 cm,102 cm
7,1/9/2017,8 cm,51 cm,104 cm
8,1/10/2017,33 cm,84 cm,127 cm
9,1/11/2017,8 cm,92 cm,122 cm


In [18]:
df_3.describe()

,Date,24 hr New Snow,Season Snowfall Total,Base Depth
count,90,90,90,90
unique,90,16,86,49
top,1/25/2016,3 cm,14 cm,147 cm
freq,1,23,2,7


In [19]:
df_4 = pd.read_csv("https://raw.githubusercontent.com/DulanDias/semantic-heterogeneity-problem/master/data_4.csv")
df_4.head(10)

,Date,24 hr New Snow,Season Snowfall Total,Base Depth
0,1-Jan-17,8 cm,8 cm,0 m
1,6-Jan-17,1 cm,9 cm,0 m
2,7-Jan-17,4 cm,13 cm,0 m
3,9-Jan-17,15 cm,28 cm,0 m
4,10-Jan-17,3 cm,31 cm,0 m
5,17-Jan-17,8 cm,39 cm,0 m
6,18-Jan-17,39 cm,78 cm,0 m
7,19-Jan-17,36 cm,114 cm,0 m
8,20-Jan-17,14 cm,128 cm,0 m
9,22-Jan-17,5 cm,133 cm,0 m


In [20]:
df_4.describe()

,Date,24 hr New Snow,Season Snowfall Total,Base Depth
count,155,155,155,155
unique,155,38,153,62
top,6-Apr-16,1 cm,298 cm,0 m
freq,1,16,2,82


Based on the above heterogenous data from multiple sources, we can identify the following variations:
* Date format
* Unit of measurement of the numerical attributes
* Null or zero values for certain numerical attributes

Note: Eventhough the attributes '24 hr New Snow', 'Season Snowfall Total' and 'Base Depth' appear to be numerical attributes, they are appended with the relevant unit of measurement.

#### **2. Define Methods required for Data Integration**

In [0]:
class HeterogeneousDataIntegrator:

  def __init__(self, *data):
      self.data = list(data)
      self.setOfColumns = set()

  def determineAttributes(self):
    print("Number of Data Sources Received: " + str(len(self.data)))
    for subset in self.data:
      for col in subset.columns: 
        self.setOfColumns.add(col)

  def dateTimeParser(self, stringDate):
    if (self.isDate(stringDate)):
      return dateutil.parser.parse(stringDate, fuzzy=True, dayfirst=True)

  def isDate(self, string, fuzzy=False):
    try: 
        dateutil.parser.parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

  def isDateTimeAttribute(self, column, fuzzy):
    #print(column)
    for value in column:
      if (not self.isDate(value, fuzzy)):
        return False
    return True

  def fit(self, fuzzy=False, upcast=True):
    self.determineAttributes()
    print("Discovered Attributes: " + str(self.setOfColumns))

    # merge multiple Data Frames in to one Panda's Data Frame
    frames = self.data
    merged = pd.concat(frames)

    # remove rows with NaN and Null values
    merged = merged.dropna()

    # standardize DateTime attributes
    for attribute in self.setOfColumns:
      if (self.isDateTimeAttribute(merged[attribute], fuzzy)):
        print(attribute + " : " + "DateTime")
        for idx, value in enumerate(merged[attribute]):
          merged[attribute][idx] = self.dateTimeParser(str(value))

    # todo: standardize Length attributes


    # todo: handling unit-less numerical measurements


    return pd.DataFrame(merged)

#### **3. Integrating Heterogeneous Data**

In [126]:
hdi = HeterogeneousDataIntegrator(df_1, df_2, df_3, df_4)

output = hdi.fit(fuzzy=False, upcast=True)
output.head(10)

Number of Data Sources Received: 4
Discovered Attributes: {'Base Depth', 'Date', '24 hr New Snow', 'Season Snowfall Total'}
Date : DateTime


,Date,24 hr New Snow,Season Snowfall Total,Base Depth
0,2017-01-02 00:00:00,8 cm,8 cm,74 cm
1,2017-01-04 00:00:00,8 cm,16 cm,71 cm
2,2017-01-05 00:00:00,8 cm,24 cm,76 cm
3,2017-01-08 00:00:00,10 cm,34 cm,81 cm
4,2017-01-10 00:00:00,20 cm,54 cm,79 cm
5,2017-01-11 00:00:00,48 cm,102 cm,99 cm
6,2017-01-12 00:00:00,10 cm,112 cm,109 cm
7,2017-01-20 00:00:00,10 cm,122 cm,94 cm
8,2017-01-21 00:00:00,3 cm,125 cm,97 cm
9,2017-01-22 00:00:00,15 cm,140 cm,94 cm
